In [1]:
import cv2
import matplotlib.pyplot as plt
import numpy as np

In [2]:
#LUCAS-KANADE Function --> Sparse Optical Flow
#accepts prev frame, prev points, next frame
#outputs points in new frame

In [3]:
#Gunner Farneback --> Dense Optical Flow

In [4]:
corner_track_params = dict(
    maxCorners=10,
    qualityLevel=0.3,
    minDistance=7,
    blockSize=7
)
corner_track_params

{'maxCorners': 10, 'qualityLevel': 0.3, 'minDistance': 7, 'blockSize': 7}

In [5]:
lk_params = dict(winSize=(200,200),
                 maxLevel=2,
                 criteria =(cv2.TERM_CRITERIA_EPS | cv2.TERM_CRITERIA_COUNT,10,0.03)
                 )

In [6]:
cap =cv2.VideoCapture(0)

ret, prev_frame = cap.read()

prev_gray = cv2.cvtColor(prev_frame, cv2.COLOR_BGR2GRAY)

#pts to track
prevPts = cv2.goodFeaturesToTrack(prev_gray,mask=None,**corner_track_params)

mask = np.zeros_like((prev_frame))# For displaying points(line) not Tracking (thats why None above)

while True:
    _,frame = cap.read()
    
    current_gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    
    nextPts, status, err = cv2.calcOpticalFlowPyrLK(prev_gray,
                                                    current_gray,
                                                    prevPts,
                                                    None,
                                                   **lk_params)
    
    good_new = nextPts[status == 1]
    good_prev = prevPts[status == 1]
    
    for i, (new,prev) in enumerate(zip(good_new,good_prev)):
        x_new, y_new = new.ravel()
        x_prev, y_prev = prev.ravel()
        
        mask = cv2.line(mask, (x_new,y_new),(x_prev,y_prev), 255, 3)

        frame = cv2.circle(frame,(x_new,y_new),8,(0,255,0), -1)

    img = cv2.add(frame,mask)
    cv2.imshow('track', img)
    
    if cv2.waitKey(1) & 0xFF == 27:
        break
    
    prev_gray = current_gray.copy()
    prevPts = good_new.reshape(-1,1,2)

cap.release()
cv2.destroyAllWindows()   

In [10]:
#DENSE

cap =cv2.VideoCapture(0)

ret, frame1 = cap.read()

prev_img = cv2.cvtColor(frame1, cv2.COLOR_BGR2GRAY)

hsv_mask = np.zeros_like(frame1)
hsv_mask[:,:,1] =255

while True:
    
    ret, frame2 = cap.read()
    next_img = cv2.cvtColor(frame2, cv2.COLOR_BGR2GRAY)
    
    flow = cv2.calcOpticalFlowFarneback(prev_img, 
                                        next_img,None, 0.5, 3, 15, 3, 5, 1.2, 0)
    
    #Convert cartesian coord to Polar Coord, since we are using hsv
    
    mag, ang = cv2.cartToPolar(flow[:,:,0],flow[:,:,1], angleInDegrees=True)
    
    hsv_mask[:,:,0] = ang/2
    
    hsv_mask[:,:,2] = cv2.normalize(mag,None,0,255,cv2.NORM_MINMAX)
    
    bgr = cv2.cvtColor(hsv_mask,cv2.COLOR_HSV2BGR)
    
    cv2.imshow('frame2',bgr)
    
    k = cv2.waitKey(30) & 0xff
    
    if k == 27:
        break
        
    prev_img = next_img
    

cap.release()
cv2.destroyAllWindows()    